In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from utils import (
    get_cross_validation_results, 
    TransactionDataset,
    TuneHyperParams,
)

from constants import Resample, Columns, ModelConstants

In [3]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.metrics import f1_score

In [4]:
data = TransactionDataset().get_training_test_split(resample=Resample.smote)

### Defining Column Transformations

In [5]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(), [Columns.SPECIFIC_HOLIDAY]),
    (log_transform, make_column_selector(dtype_include="number"))
)

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.SPECIFIC_HOLIDAY, Columns.CUSTOMER_TYPE]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

In [6]:
regression_model = LogisticRegression(
    random_state=ModelConstants.RANDOM_STATE,
    max_iter=ModelConstants.MAX_ITERATIONS * 10,
    warm_start=True
)

In [7]:
log_and_ordinal_model_pipeline = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("logistic_regression", regression_model)
    ]
)

norm_and_one_hot_model_pipeline = Pipeline(
    [
        ("column_transformation", norm_and_one_hot_transforms),
        ("logistic_regression", regression_model)
    ]
)

In [8]:
tuning_params = {
    "logistic_regression__penalty": ["l1", "l2"],
    "logistic_regression__solver": ["saga", "liblinear"],
    "logistic_regression__C": [0.1, 0.5, 1, 1.5, 2, 2.5, 3],
    "logistic_regression__class_weight": [
        "balanced",
        {1: 0.55, 0: 0.45},
        {1: 0.6, 0: 0.4},
        {1: 0.65, 0: 0.35},
        {1: 0.7, 0: 0.3},
        {1: 0.75, 0: 0.25},
        {1: 0.8, 0: 0.2},
    ],
}

In [9]:
gs = (
    TuneHyperParams()
    .full_grid_search(
     log_and_ordinal_model_pipeline,
     tuning_params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_scores_and_params()
)

{'logistic_regression__C': 0.1, 'logistic_regression__class_weight': {1: 0.6, 0: 0.4}, 'logistic_regression__penalty': 'l2', 'logistic_regression__solver': 'liblinear'}
Best parameter (CV score: 0.876):


In [10]:
gs = (
    TuneHyperParams()
    .full_grid_search(
     norm_and_one_hot_model_pipeline,
     tuning_params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_scores_and_params()
)

{'logistic_regression__C': 3, 'logistic_regression__class_weight': {1: 0.55, 0: 0.45}, 'logistic_regression__penalty': 'l1', 'logistic_regression__solver': 'liblinear'}
Best parameter (CV score: 0.818):


### TEST Score

In [16]:
final_log_model = LogisticRegression(
    random_state=ModelConstants.RANDOM_STATE,
    max_iter=ModelConstants.MAX_ITERATIONS * 10,
    class_weight={1: 0.6, 0: 0.4},
    penalty="l2",
    C=0.1,
    solver="liblinear"
)

final_norm_model = LogisticRegression(
    random_state=ModelConstants.RANDOM_STATE,
    max_iter=ModelConstants.MAX_ITERATIONS * 10,
    class_weight={1: 0.55, 0: 0.45},
    penalty="l1",
    C=3,
    solver="liblinear"
)

In [17]:
final_log_model_pipeline = Pipeline(
    [
        ("column_transformations", log_and_ordinal_transforms),
        ("logistic_regression", final_log_model)
    ]
)

final_norm_model_pipeline = Pipeline(
    [
        ("column_transformations", log_and_ordinal_transforms),
        ("logistic_regression", final_norm_model)
    ]
)

In [21]:
get_cross_validation_results(
    final_log_model_pipeline,
    data.TRAINING.predictors,
    data.TRAINING.outcome
);
print("="*100)
get_cross_validation_results(
    final_norm_model_pipeline,
    data.TRAINING.predictors,
    data.TRAINING.outcome
);

fit_time: 0.06392292976379395
score_time: 0.009336757659912109
test_accuracy: 0.8789783277627461
test_balanced_accuracy: 0.8789755099151814
test_f1: 0.877598760917909
fit_time: 0.6096999406814575
score_time: 0.00855720043182373
test_accuracy: 0.8774002525922387
test_balanced_accuracy: 0.8773973976335689
test_f1: 0.8750072066734464


In [19]:
final_log_model_pipeline.fit(
    data.TRAINING.predictors,
    data.TRAINING.outcome
)

predictions = final_log_model_pipeline.predict(
    data.TESTING.predictors
)

f1_score(
    predictions, data.TESTING.outcome
)

0.6614525139664804

In [20]:
final_norm_model_pipeline.fit(
    data.TRAINING.predictors,
    data.TRAINING.outcome
)

predictions = final_norm_model_pipeline.predict(
    data.TESTING.predictors
)

f1_score(
    predictions, data.TESTING.outcome
)

0.6590389016018307